## People Centered COS
## CDOT Data - El Paso County 
## Basic Map with CUIDs

### Set Up

In [17]:
pip install pandas -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install openpyxl -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install geopandas -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install matplotlib -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install folium -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install branca -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install h3 -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
import matplotlib.pyplot as plt

import folium
from folium import plugins
import matplotlib.cm as cm 
import matplotlib.colors as colors
from branca.element import Element

import h3

### Load Data

In [25]:
import sys  
sys.path.insert(1, '../')

from cdot_clean_geography_columns import handle_missing_values_in_geography_columns

In [57]:
# Load CDOT Data for a list of years

years = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

cdot_base_path = "/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/2_fill_lat_long_elpaso_w_uuid_KJS/"

loaded_data_list = []
for y in years:
    filename = f"CDOT_ElPaso_Crash_{y}.csv"
    full_path = cdot_base_path + filename
    print(full_path)
    year_cdot_pdf = pd.read_csv(cdot_base_path + filename)
    year_cdot_pdf['Crash Year'] = y
    print(list(year_cdot_pdf.columns)[0:5])
    print(year_cdot_pdf.shape)

    loaded_data_list.append(year_cdot_pdf) 
    
cdot_pdf_load = pd.concat(loaded_data_list)

/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/2_fill_lat_long_elpaso_w_uuid_KJS/CDOT_ElPaso_Crash_2007.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(11560, 143)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/2_fill_lat_long_elpaso_w_uuid_KJS/CDOT_ElPaso_Crash_2008.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(9955, 143)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/2_fill_lat_long_elpaso_w_uuid_KJS/CDOT_ElPaso_Crash_2009.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(10322, 143)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/2_fill_lat_long_elpaso_w_uuid_KJS/CDOT_ElPaso_Crash_2010.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(10107, 143)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/2_fill_lat_long_elpaso_w_uuid_KJS/CDOT_ElPaso_Crash_2011.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(10085, 143)
/

## Prepare and Clean Data

In [60]:
# Drop duplicates
print('Data size after loading', cdot_pdf_load.shape)
cdot_pdf_withNans_dedup = cdot_pdf_load.drop_duplicates()
print('Data size after drop duplicates', cdot_pdf_withNans_dedup.shape)
#display(cdot_pdf_withNans_dedup.head(5))

Data size after loading (196216, 143)
Data size after drop duplicates (196216, 143)


In [61]:
# Select columns for analysis 
analysis_columns = [
    'CUID',
    'Crash Date', 
    'City', 
    'County',
    'Latitude', 
    'Longitude',
    'First HE', 
    'Second HE', 
    'Third HE', 
    'Fourth HE', 
    'MHE', 
    'Crash Type', 
    'TU-1 NM Type', 
    'TU-2 NM Type',
    'TU-3 NM Type',
]

cdot_pdf = cdot_pdf_withNans_dedup[analysis_columns]

# check NaN count by column 
print('nan count before handling') 
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])
print()

# handle nan, null, and other corner cases
cdot_pdf = handle_missing_values_in_geography_columns(cdot_pdf)

# check NaN count by column 
print('nan count after handling')
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])
print()

# Fix CUID 322316 
cdot_pdf.loc[cdot_pdf['Longitude'].astype(int) == 104, 'Longitude'] = -1*cdot_pdf.loc[cdot_pdf['Longitude'].astype(int) == 104, 'Longitude'].values[0]

# Drop any latitude and longitude values that are 999
cdot_pdf_good_LL = cdot_pdf[(cdot_pdf['Latitude'].astype(int) != 999) & (cdot_pdf['Longitude'].astype(int) != 999)]
print('count after removing invalid lat long', cdot_pdf_good_LL.shape)

nan count before handling
CUID 0
CUID 0
Crash Date 0
Crash Date 0
City 38256
City 38256
County 0
County 0
Latitude 495
Latitude 495
Longitude 495
Longitude 495
First HE 30767
First HE 30767
Second HE 166157
Second HE 166157
Third HE 191721
Third HE 191721
Fourth HE 195514
Fourth HE 195514
MHE 30847
MHE 30847
Crash Type 7839
Crash Type 7839
TU-1 NM Type 194397
TU-1 NM Type 194397
TU-2 NM Type 193939
TU-2 NM Type 193939
TU-3 NM Type 196142
TU-3 NM Type 196142

nan count after handling
CUID 0
CUID 0
Crash Date 0
Crash Date 0
City 0
City 0
County 0
County 0
Latitude 0
Latitude 0
Longitude 0
Longitude 0
First HE 30767
First HE 30767
Second HE 166157
Second HE 166157
Third HE 191721
Third HE 191721
Fourth HE 195514
Fourth HE 195514
MHE 30847
MHE 30847
Crash Type 7839
Crash Type 7839
TU-1 NM Type 194397
TU-1 NM Type 194397
TU-2 NM Type 193939
TU-2 NM Type 193939
TU-3 NM Type 196142
TU-3 NM Type 196142

count after removing invalid lat long (192022, 15)


In [62]:
gdf = gpd.GeoDataFrame(
    cdot_pdf_good_LL, 
    geometry=gpd.points_from_xy(
        cdot_pdf_good_LL['Longitude'],
        cdot_pdf_good_LL['Latitude']
          )
        )

# have to set the coordinate reference system for map to render
# epsg code - integer or string 
gdf = gdf.set_crs(epsg=4326)
gdf = gdf.to_crs("EPSG:4326")

display(gdf.head())

,CUID,Crash Date,City,County,Latitude,Longitude,First HE,Second HE,Third HE,Fourth HE,MHE,Crash Type,TU-1 NM Type,TU-2 NM Type,TU-3 NM Type,geometry
0,4ed1037c-8377-4130-a3a4-ff931ec54689,1/1/2007,COLORADO SPRINGS,EL PASO,38.931747,-104.741930,Light Pole/Utility Pole,NaN,NaN,NaN,Light Pole/Utility Pole,Light Pole/Utility Pole,NaN,NaN,NaN,POINT (-104.74193 38.93175)
1,e4538035-41b1-4172-baa7-ad54707a8b9e,1/1/2007,COLORADO SPRINGS,EL PASO,38.862794,-104.745780,Curb,Other Fixed Object (Describe in Narrative),NaN,NaN,Other Fixed Object (Describe in Narrative),Curb,NaN,NaN,NaN,POINT (-104.74578 38.86279)
2,fb919c2e-9c47-46b1-994c-1ef73058f0be,1/1/2007,COLORADO SPRINGS,EL PASO,38.846313,-104.720786,old__broadside,NaN,NaN,NaN,old__broadside,Broadside,NaN,NaN,NaN,POINT (-104.72079 38.84631)
3,0e873507-f3a9-4860-89e5-85d5c4a427e1,1/1/2007,MANITOU SPRINGS,EL PASO,38.862410,-104.917830,Parked Motor Vehicle,NaN,NaN,NaN,Parked Motor Vehicle,Parked Motor Vehicle,NaN,NaN,NaN,POINT (-104.91783 38.86241)
4,c02e9cd9-f798-4d03-ab18-4b46f1604fde,1/1/2007,COLORADO SPRINGS,EL PASO,38.966677,-104.784641,Overturning/Rollover,NaN,NaN,NaN,Overturning/Rollover,Overturning/Rollover,NaN,NaN,NaN,POINT (-104.78464 38.96668)


In [63]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Create Map with All Crashes 2020-2024

In [64]:
# interactive Open Street Map map
#gdf.explore()

In [65]:
# Save the explore map to a file 
#m = gdf.explore()
#m.save("cdot_crash_elpaso_2020_to_2024_cuids.html")

### Limit to Bike and Pedestrian Crashes in El Paso County 

In [66]:
# bpep - bike, ped el paso
# Harmful events 
# Crash Type 
# Non motorist 
he_list = ['Pedestrian', 'Bicycle/Motorized Bicycle']
tunm_list = ['Pedestrian', 'Bicyclist', 'Other Pedestrian', 'Wheelchair', 'Scooter']
bike_ped_condition = (gdf['First HE'].isin(he_list)) | \
                     (gdf['Second HE'].isin(he_list)) | \
                     (gdf['Third HE'].isin(he_list)) | \
                     (gdf['Fourth HE'].isin(he_list)) | \
                     (gdf['MHE'].isin(he_list)) | \
                     (gdf['Crash Type'].isin(he_list)) | \
                     (gdf['TU-1 NM Type'].isin(tunm_list)) | \
                     (gdf['TU-2 NM Type'].isin(tunm_list)) | \
                     (gdf['TU-3 NM Type'].isin(tunm_list))

cdot_gdf_bpep = gdf.loc[bike_ped_condition]
print('CDOT El Paso bike and ped crashes size', cdot_gdf_bpep.shape)

CDOT El Paso bike and ped crashes size (4307, 16)


In [75]:
# Additional Cleaning 
# two crashes out of Colorado (New Mexico and Texas) 
cdot_gdf_bpep_filt = cdot_gdf_bpep.loc[cdot_gdf_bpep['Latitude'] > 37]

In [76]:
cdot_gdf_bpep_filt_slim_cols = cdot_gdf_bpep_filt[['CUID', 'Crash Date', 'City', 'County', 'Latitude', 'Longitude']]

In [78]:
gdf_bikeped_cleaned = gpd.GeoDataFrame(
    cdot_gdf_bpep_filt_slim_cols, 
    geometry=gpd.points_from_xy(
        cdot_gdf_bpep_filt_slim_cols['Longitude'],
        cdot_gdf_bpep_filt_slim_cols['Latitude']
          )
        )

# have to set the coordinate reference system for map to render
# epsg code - integer or string 
gdf_bikeped_cleaned = gdf_bikeped_cleaned.set_crs(epsg=4326)
gdf_bikeped_cleaned = gdf_bikeped_cleaned.to_crs("EPSG:4326")

display(gdf_bikeped_cleaned.head())

,CUID,Crash Date,City,County,Latitude,Longitude,geometry
16,8ef1a645-5d56-4e92-81a4-417897bd57e6,1/1/2007,COLORADO SPRINGS,EL PASO,38.851164,-104.757322,POINT (-104.75732 38.85116)
21,e991b79f-f4e8-41df-bda5-4b9112d57515,1/1/2007,COLORADO SPRINGS,EL PASO,38.796424,-104.755930,POINT (-104.75593 38.79642)
78,6e2b1692-822d-4928-bfdd-2432c8c3610b,1/3/2007,COLORADO SPRINGS,EL PASO,38.852814,-104.794200,POINT (-104.7942 38.85281)
109,0928cbf4-2fb9-48fa-9ef4-698363c2840f,1/3/2007,NONE,EL PASO,38.768084,-104.739270,POINT (-104.73927 38.76808)
148,26a62fdb-fc0c-4c09-98b7-828e8e6dc74e,1/4/2007,COLORADO SPRINGS,EL PASO,38.873116,-104.819631,POINT (-104.81963 38.87312)


In [79]:
gdf_bikeped_cleaned.explore()

In [80]:
# Save the explore map to a file 
m = gdf_bikeped_cleaned.explore()
m.save("cdot_crash_elpaso_bikeped_2007_to_2024_cuids.html")